Check GPU

In [1]:
!nvidia-smi

Sat May 28 13:10:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    47W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Load Google Drive 

In [2]:
from google.colab import drive #Only if you are using Google Drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive
Mounted at /content/gdrive
/content/gdrive/My Drive


Location of spectograms for home location, small sample testing

In [4]:
cd /content/gdrive/MyDrive/content/img_dir/us8k_freesound/

/content/gdrive/MyDrive/content/img_dir/us8k_freesound


In [5]:
%tensorflow_version  1.x

TensorFlow 1.x selected.


**Model**

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

training_dir = '/content/gdrive/MyDrive/content/img_dir/us8k_freesound/train/' 
validation_dir = '/content/gdrive/MyDrive/content/img_dir/us8k_freesound/validation/'
test_dir = '/content/gdrive/MyDrive/content/img_dir/us8k_freesound/test'

folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 206


Import Libraries

In [7]:
# importing requried libraries
from keras import applications
from keras.models import Model
import keras 
from keras.applications.vgg16 import VGG16 #Importing the VGG16 Model
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D,BatchNormalization
from keras.applications import VGG16, MobileNetV2
from keras.optimizers import SGD
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input  

Using TensorFlow backend.


**Data pre-processing**

In [9]:
IMAGE_SIZE = [224,224]

training_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=40, 
                                      width_shift_range=0.3, height_shift_range=0.3, 
                                      shear_range=0.3, horizontal_flip=True, fill_mode="nearest",
                                      preprocessing_function=preprocess_input)
training_generator = training_datagen.flow_from_directory(directory=training_dir,target_size = IMAGE_SIZE, batch_size = 255, class_mode = 'categorical')
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(directory=test_dir, target_size = IMAGE_SIZE, batch_size = 255, class_mode = 'categorical')
training_generator.class_indices


Found 43690 images belonging to 206 classes.
Found 5911 images belonging to 189 classes.


{'Accelerating_and_revving_and_vroom': 0,
 'Accordion': 1,
 'Acoustic_guitar': 2,
 'Aircraft': 3,
 'Alarm': 4,
 'Animal': 5,
 'Applause': 6,
 'Bark': 7,
 'Bass_drum': 8,
 'Bass_guitar': 9,
 'Bathtub_filling_or_washing': 10,
 'Bell': 11,
 'Bicycle': 12,
 'Bicycle_bell': 13,
 'Bird': 14,
 'Bird_vocalization_and_bird_call_and_bird_song': 15,
 'Boat_and_Water_vehicle': 16,
 'Boiling': 17,
 'Boom': 18,
 'Bowed_string_instrument': 19,
 'Brass_instrument': 20,
 'Breathing': 21,
 'Burping_and_eructation': 22,
 'Bus': 23,
 'Buzz': 24,
 'Camera': 25,
 'Car': 26,
 'Car_passing_by': 27,
 'Cat': 28,
 'Chatter': 29,
 'Cheering': 30,
 'Chewing_and_mastication': 31,
 'Chicken_and_rooster': 32,
 'Child_speech_and_kid_speaking': 33,
 'Chime': 34,
 'Chink_and_clink': 35,
 'Chirp_and_tweet': 36,
 'Chuckle_and_chortle': 37,
 'Church_bell': 38,
 'Clapping': 39,
 'Clock': 40,
 'Coin_dropping': 41,
 'Computer_keyboard': 42,
 'Conversation': 43,
 'Cough': 44,
 'Cowbell': 45,
 'Crack': 46,
 'Crackle': 47,
 'Cra

Load VGG16

In [10]:
vgg_model = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  

for layer in vgg_model.layers:
      layer.trainable = False # Non trainable weights

# Create Dense Layers
# Add the last layers (Flatten and Dense layers) for our problem
x = Flatten()(vgg_model.output) 
x = Dense(num_classes, activation = 'softmax')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting

transfer_model = Model(inputs = vgg_model.input, outputs = x)
# Compile model, for this we will be using ADAM optimiser to reach to the global minima while training our model
# learning_rate= 5e-5
transfer_model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
#history = transfer_model.fit(X_train, y_train, batch_size = 1, epochs=50, validation_data=(X_test,y_test))

#transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
transfer_model.summary() #check summary of the model using this command


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

58892288/58889256 [==============================] - 1s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112

In [11]:
history = transfer_model.fit_generator(training_generator,
                   steps_per_epoch = 255, #Number of iterations = number of training images (4055) / batch size (255)  
                   epochs = 50, 
                   validation_data = validation_generator,
                   validation_steps = 2, #same for validation data 300 validation images
                   shuffle = True) 
transfer_model.save('/content/gdrive/MyDrive/loc_home/us8k_freesound_vgg16_home_255_Param.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/50


/tensorflow-1.15.2/python3.7/keras/utils/data_utils.py:616: UserWarning: The input 150 could not be retrieved. It could be because a worker has died.
  UserWarning)


 12/255 [>.............................] - ETA: 7:46:39 - loss: 8.5689 - accuracy: 0.0059

/tensorflow-1.15.2/python3.7/keras/utils/data_utils.py:616: UserWarning: The input 136 could not be retrieved. It could be because a worker has died.
  UserWarning)


 13/255 [>.............................] - ETA: 7:26:22 - loss: 7.9097 - accuracy: 0.0054

...

/tensorflow-1.15.2/python3.7/keras/utils/data_utils.py:616: UserWarning: The input 61 could not be retrieved. It could be because a worker has died.
  UserWarning)


171/255 [===================>..........] - ETA: 1:21:47 - loss: 0.6013 - accuracy: 0.0018

/tensorflow-1.15.2/python3.7/keras/utils/data_utils.py:616: UserWarning: The input 86 could not be retrieved. It could be because a worker has died.
  UserWarning)


254/255 [============================>.] - ETA: 40s - loss: 0.4048 - accuracy: 0.0018 

ValueError: ignored

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
from keras.models import load_model
model = load_model('/content/gdrive/MyDrive/loc_home/us8k_freesound_vgg16_home_255_Param.h5')

In [ ]:
!pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 15.7 MB 30.2 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.5.3 which is incompatible.
lucid 0.3.10 requires numpy<=1.19, but you have numpy 1.21.6 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, bu

Model Testing

In [ ]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os
img_size = 224
dir_loc ='/content/gdrive/MyDrive/loc_home/dataset/test'
clssN = os.listdir(dir_loc)

#clssN = ['Heidelberglaan15','Padualaan101','Padualaan97']
count = 0
countT = 0
for bl in clssN:
  src = './dataset/test/' + bl 
  imgs = os.listdir(src)
  count = 0
  countT = len(imgs) #total number of images per class
  for img in imgs: 
    im = load_img(src + "/" + img)
    w,h = im.size
    im = im.resize((int(w*0.2),int(h*0.2)))
    im = im.resize((img_size,img_size))
    x = img_to_array(im)  
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    x/=255.
    probs = transfer_model.predict(x, verbose=0)
    maxDet = max(probs[0])
    maxIdx = list(probs[0]).index(maxDet) #predicted class with maximum probability
    if (clssN[maxIdx]==bl): 
      count = count + 1  #count for true predictions (true positives)
    else:
      print(f'Wrong prediction of {bl}  {img}. Predicted as {clssN[maxIdx]}') 
  print(f'test accuracy for {bl} is {str(count/countT)}')

IndexError: ignored

Using the below code, you will see the output of what building class the model has predicted for this specific image.

Plotting loss and accuracy